In [22]:
#Get monomials in k[x,y,z][d]
def getEntries(d):
    lis = []
    #solutions to a+b+c=d
    for a in range(d+1):
        for b in range(d-a+1):
            lis.append((a,b,d-a-b))
    return lis
        

In [25]:
#Gets entries for the matrix/polyomial multiplicaiton between k[x,y,z][d-2m-1] and k[x,y,z][d]
#i,j are the row and column of the entry
def get_entries_s12(d, m, i,j):
    coe = []
    mon = getEntries(d-2*m-1)[j]
    new = getEntries(d)[i]
    (x,y,z) = mon
    (t,w,r) = new
    dif = (t-x,w-y,r-z)
    (a,b,c) = dif
    #if the difference is a monomial of (x-y)^(2m+1), then the entry is the coeffecient of that monomial
    if c==0 and a+b==2*m+1 and a>-1 and b>-1:
        entry = factorial(2*m+1)/factorial(a)/factorial(b)
        return entry
    else:
        return 0
def get_entries_s13(d, m, i,j):
    coe = []
    mon = getEntries(d-2*m-1)[j]
    new = getEntries(d)[i]
    (x,y,z) = mon
    (t,w,r) = new
    dif = (t-x,w-y,r-z)
    (a,b,c) = dif
    if b==0 and a+c==2*m+1 and a>-1 and c>-1:
        entry = factorial(2*m+1)/factorial(a)/factorial(c)
        return entry
    else:
        return 0
def get_entries_s23(d, m, i,j):
    coe = []
    mon = getEntries(d-2*m-1)[j]
    new = getEntries(d)[i]
    (x,y,z) = mon
    (t,w,r) = new
    dif = (t-x,w-y,r-z)
    (a,b,c) = dif
    if a==0 and b+c==2*m+1 and b>-1 and c>-1:
        entry = factorial(2*m+1)/factorial(b)/factorial(c)
        return entry
    else:
        return 0

        
    
    

In [26]:
#contruct the matrix representing multiplicaiton by (x-y)^(2m+1)
def getMultiS12(d,m):
    A = matrix(GF(3),len(getEntries(d)),len(getEntries(d-2*m-1)), lambda i, j: get_entries_s12(d, m, i, j))
    
    return A
def getMultiS13(d,m):
    A = matrix(GF(3),len(getEntries(d)),len(getEntries(d-2*m-1)), lambda i, j: get_entries_s13(d, m, i, j))
    
    return A
def getMultiS23(d,m):
    A = matrix(GF(3),len(getEntries(d)),len(getEntries(d-2*m-1)), lambda i, j: get_entries_s23(d, m, i, j))
    
    return A
    

In [27]:
#entries for the matrix of multiplication be s12
def get_s12_entr(d,i,j):
    (a,b,c) = getEntries(d)[j]
    new = (b,a,c)
    if getEntries(d)[i] == new:
        return 1
    else:
        return 0
def get_s13_entr(d,i,j):
    (a,b,c) = getEntries(d)[j]
    new = (c,b,a)
    if getEntries(d)[i] == new:
        return 1
    else:
        return 0
def get_s23_entr(d,i,j):
    (a,b,c) = getEntries(d)[j]
    new = (a,c,b)
    if getEntries(d)[i] == new:
        return 1
    else:
        return 0
    

In [28]:
#Constructs matrix s12
def getS12(d):
    length = len(getEntries(d))
    A = matrix(GF(3),length,length, lambda i, j: get_s12_entr(d, i, j))
    
    return A

def getS13(d):
    length = len(getEntries(d))
    A = matrix(GF(3),length,length, lambda i, j: get_s13_entr(d, i, j))
    
    return A  

def getS23(d):
    length = len(getEntries(d))
    A = matrix(GF(3),length,length, lambda i, j: get_s23_entr(d, i, j))
    
    return A  


In [29]:
def isEqual(i,j):
    if(i==j):
        return 1
    else:
        return 0

In [30]:
#contructs identity matrix
def getIdentity(d):
    length = len(getEntries(d))
    A= matrix(GF(3),length,length, lambda i, j: isEqual(i,j))
    return A

In [31]:
#matrix for (1-s12)
def subtractS12(d):
    return getIdentity(d) - getS12(d)
def subtractS13(d):
    return getIdentity(d) - getS13(d)
def subtractS23(d):
    return getIdentity(d) - getS23(d)

In [59]:
#finds the preimage of the multiples of (x-y)^(2m+1) for (1-s12).
def quasiS12(d,m):
    #gets the space of multiples of (x-y)^(2m+1) in degree d
    V = getMultiS12(d,m).column_space()
    #obtains intersection of multiples and the image of (1-s12)
    inter = V.intersection(subtractS12(d).column_space())
    basis_inter = inter.basis()
    new_basis = []
    for i in range(len(basis_inter)):
        try:
            #for each basis element, find the preimage of it by solving the matrix
            q = subtractS12(d).solve_right(basis_inter[i])
            new_basis.append(q)
        except Exception:
            n=1+1;
    #add the kernel of (1-s12) to get the entire preimage
    ker = subtractS12(d).kernel()
    ker_basis = ker.basis()
    return span(new_basis+ ker_basis)
def quasiS13(d,m):
    V = getMultiS13(d,m).column_space()
    inter = V.intersection(subtractS13(d).column_space())
    basis_inter = inter.basis()
    new_basis = []
    for i in range(len(basis_inter)):
        try:
            q = subtractS13(d).solve_right(basis_inter[i])
            new_basis.append(q)
        except Exception:
            n=1+1;
    ker = subtractS13(d).kernel()
    ker_basis = ker.basis()
    return span(new_basis+ ker_basis)
def quasiS23(d,m):
    V = getMultiS23(d,m).column_space()
    inter = V.intersection(subtractS23(d).column_space())
    basis_inter = inter.basis()
    new_basis = []
    for i in range(len(basis_inter)):
        try:
            q = subtractS23(d).solve_right(basis_inter[i])
            new_basis.append(q)
        except Exception:
            n=1+1;
    ker = subtractS23(d).kernel()
    ker_basis = ker.basis()
    return span(new_basis+ ker_basis)

In [60]:
#Takes the intersection of each of the preimage of the multiples of (x-y)^(2m+1) for (1-s12).
def getQuasi(d,m):
    V1 = quasiS12(d,m)
    V2 = quasiS13(d,m)
    V3 = quasiS23(d,m)
    interm = V1.intersection(V2)
    final = V3.intersection(interm)
    return final

In [40]:
#Finds the dimension of th esymmetric polynomials in degree d
def getSymDeg(d):
    num=0
    for a in range(d+1):
        for b in range(floor((d-a)/2)+1):

            if (d-a-2*b) % 3 == 0:
                num=num+1
    return num